In [11]:
from cognite.client.data_classes import FileMetadataWrite
from cognite.client.data_classes.data_modeling import SpaceApply
from cognite.client.data_classes.data_modeling.cdm.v1 import CogniteFile, CogniteFileApply
from cognite.client.exceptions import CogniteException

from tests.auth_utils import get_toolkit_client

In [2]:
client = get_toolkit_client("dev-cluster.env", enable_set_pending_ids=True)

Found dev-cluster.env file in repository root. Loaded variables from dev-cluster.env file.


In [3]:
SPACE = "sp_upgrade"

In [4]:
client.data_modeling.spaces.apply(SpaceApply(space=SPACE))

,value
space,sp_upgrade
is_global,False
last_updated_time,2025-07-04 09:38:40.297000
created_time,2025-07-04 09:38:40.297000


# Mismatch Cognite/Asset-Centric

In [7]:
file = FileMetadataWrite(
    external_id="ts_case3_mismatch",
    name="Case 3 Mismatch Cognite-Asset-Centric",
)

In [8]:
created, _ = client.files.create(file)
created

,value
external_id,ts_case3_mismatch
name,Case 3 Mismatch Cognite-Asset-Centric
labels,[]
id,8852703828481655
created_time,2025-07-10 05:39:29.763000
last_updated_time,2025-07-10 05:39:29.763000
uploaded,False


In [9]:
filecontent = b"This is filecontent"

In [10]:
client.files.upload_content_bytes(filecontent, external_id=created.external_id)

,value
external_id,ts_case3_mismatch
name,Case 3 Mismatch Cognite-Asset-Centric
id,8852703828481655
created_time,2025-07-10 05:39:29.763000
last_updated_time,2025-07-10 05:39:29.763000
uploaded,False


In [12]:
try:
    updated = client.files.set_pending_ids((SPACE, file.external_id), external_id=file.external_id)
except CogniteException as e:
    print(f"\033[91mError\033[0m {e.code} {e.message!s}")
else:
    updated

In [13]:
cognite_file = CogniteFileApply(
    space=SPACE,
    external_id=file.external_id,
    name="Another name for Case 3 Mismatch",
)

In [14]:
created_dm = client.data_modeling.instances.apply(cognite_file).nodes
created_dm

,space,instance_type,external_id,version,was_modified,last_updated_time,created_time
0,sp_upgrade,node,ts_case3_mismatch,1,True,2025-07-10 05:39:54.250,2025-07-10 05:39:54.250


In [16]:
retrieved_file = client.files.retrieve(external_id=file.external_id)

In [17]:
retrieved_file

,value
external_id,ts_case3_mismatch
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 3 Mismatch Cognite-Asset-Centric
id,8852703828481655
created_time,2025-07-10 05:39:29.763000
last_updated_time,2025-07-10 05:39:44.822000
uploaded,True
uploaded_time,2025-07-10 05:39:32.543000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."


In [18]:
retrieved_cognite_file = client.data_modeling.instances.retrieve_nodes(cognite_file.as_id(), node_cls=CogniteFile)

In [19]:
retrieved_cognite_file

,value
space,sp_upgrade
external_id,ts_case3_mismatch
version,1
last_updated_time,2025-07-10 05:39:54.250000
created_time,2025-07-10 05:39:54.250000
instance_type,node
name,Another name for Case 3 Mismatch
description,None
tags,None
aliases,None


In [20]:
retrieved_cognite_ts_classic = client.files.retrieve(instance_id=cognite_file.as_id())
retrieved_cognite_ts_classic

,value
external_id,ts_case3_mismatch
instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
name,Case 3 Mismatch Cognite-Asset-Centric
id,8852703828481655
created_time,2025-07-10 05:39:29.763000
last_updated_time,2025-07-10 05:39:44.822000
uploaded,True
uploaded_time,2025-07-10 05:39:32.543000
pending_instance_id,"{'space': 'sp_upgrade', 'external_id': 'ts_cas..."
